# CH01.2. **역전파(Back propagation)**

## 00. **작업 환경 설정**

#### 00.0. **사전 변수 설정**

In [1]:
seed_num = 2025

#### 00.1. **라이브러리 호출 및 옵션 설정**

In [2]:
#(1) Import libraries
import numpy as np
import torch

In [3]:
#(2) Set options
np.random.seed(seed=seed_num)
torch.manual_seed(seed=seed_num)
torch.cuda.manual_seed_all(seed=seed_num)

In [4]:
#(3) Define user-defined function
pass

In [5]:
#(4) Define class
pass

<b></b>

## 01. **자동 미분(Automatic Differentiation)**

#### 01.1. **정의** : 자동 미분(automatic differentiation)을 수행해주는 핵심 엔진

#### 01.2. **동작 원리** :
#### $ \hspace{0.15cm} $ ① `requires_grad`가 설정된 텐서가 연산에 참여하게 되면, PyTorch는 내부적으로 연산 그래프(Computation Graph)를 구성
#### $ \hspace{0.15cm} $ ② 각 연산(operation)에 대한 Function 객체가 생성되어 그래프에 연결
#### $ \hspace{0.15cm} $ ③ 손실 함수를 최종적으로 계산한 뒤, `.backward()` 메서드를 호출하면, 계산 그래프를 거슬러 올라가면서(Backprop) 기울기를 자동 연산

<b></b>

## 02. **연산 그래프(Computation Graph)**

#### 02.1. **정의** : 수학적 연산을 노드(node)와 간선(edge)로 연결해 나타낸 그래프 구조

#### 01.2. **자동 미분에서의 특징** :
#### $ \hspace{0.15cm} $ ① 데이터의 및 실행된 모든 연산들의 기록을 객체로 구성된 방향성 비순환 그래프(DAG; Directed Acyclic Graph)에 저장함
#### $ \hspace{0.15cm} $ ② 순전파 단계에서 요청된 연산에 따라 결과를 계산하고, DAG에 gradient function을 저장함
#### $ \hspace{0.15cm} $ ③ 역전파 단계에서 각 gradient function을 통해 gradient를 계산 및 연쇄 법칙을 이용함

<b></b>

## 03. **역전파(Back Propagation)**

#### **[GRAPH]**

#### 03.1. 자동미분 준비 : `torch.tensor(, requires_grad=True)`

In [6]:
#(1)
x = torch.ones(size=(2, 2), dtype=torch.float64, requires_grad=True)

#(2)
x

tensor([[1., 1.],
        [1., 1.]], dtype=torch.float64, requires_grad=True)

In [7]:
#(3)
y = x + 1 

#(4)
y

tensor([[2., 2.],
        [2., 2.]], dtype=torch.float64, grad_fn=<AddBackward0>)

In [8]:
#(5)
z = 2 * y ** 2

#(6) 
z

tensor([[8., 8.],
        [8., 8.]], dtype=torch.float64, grad_fn=<MulBackward0>)

In [9]:
#(7)
obj_func = z.mean()

#(9)
obj_func

tensor(8., dtype=torch.float64, grad_fn=<MeanBackward0>)

#### 03.2. 역전파 시행 : `torch.tensor.grad`

In [10]:
#(1) Perform backpropagation based on `obj_func`
obj_func.backward()

In [11]:
#(2)
x.grad

tensor([[2., 2.],
        [2., 2.]], dtype=torch.float64)

#### **(`PLUS`)** 이를 수식으로 표현하면 아래와 같음
#### $ \text{obj\_func} = \displaystyle\sum^{2}_{i=1}\displaystyle\sum^{2}_{k=1}\frac{z_{i,k}}{4} = \frac{z_{1,1} + z_{1,2} + z_{2,1} + z_{2,2}}{4} $ 
#### $ \hspace{1.45cm} = \frac{2 (y_{1,1})^{2} + 2 (y_{1,2})^{2} + 2 (y_{2,1})^{2} + 2(y_{2,2})^{2}}{4} $
#### $ \hspace{1.45cm} = \frac{2 (x_{1,1} + 1)^{2} + 2 (x_{1,2} + 1)^{2} + 2 (x_{2,1} + 1)^{2} + 2 (x_{2,2} + 1)^{2}}{4} $
#### $ \hspace{1.45cm} = \frac{(x_{1,1} + 1)^{2} + (x_{1,2} + 1)^{2} + (x_{2,1} + 1)^{2} + (x_{2,2} + 1)^{2}}{2} $
#### $ \nabla{}_{\textbf{x}} (\text{obj\_func}) = \frac{\partial{}}{\partial{}\textbf{x}} \text{obj\_func} = \begin{bmatrix} \frac{\partial{}(\text{obj\_func})}{\partial{}x_{1,1}} & \frac{\partial{}(\text{obj\_func})}{\partial{}x_{1,2}} \\ \frac{\partial{}(\text{obj\_func})}{\partial{}x_{2,1}} & \frac{\partial{}(\text{obj\_func})}{\partial{}x_{2,2}} \end{bmatrix} = \begin{bmatrix} x_{1,1}+1 & x_{1,2}+1 \\ x_{2,1}+1 & x_{2,2}+1 \end{bmatrix} \;\; $ ($ \because{} \, \textbf{x} = \begin{bmatrix} x_{1,1} & x_{1,2} \\ x_{2,1} & x_{2,2} \end{bmatrix} $)
#### $ \therefore{} \text{ if } \, x_{i,k} = 1, \;\; \frac{\partial{}}{\partial{}\textbf{x}} \text{obj\_func} = \begin{bmatrix} 2 & 2 \\ 2 & 2 \end{bmatrix} $

#### **(`PLUS`)** 실제로 pytorch에서는 미분의 체인룰을 사용하여 계산함